In [4]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [20]:
import cv2
import os 
from pytube import YouTube
from skimage.metrics import structural_similarity as ssim

In [13]:
import yt_dlp
import os

def download_youtube_video_ytdlp(url: str, output_path: str = "videos") -> str:
    os.makedirs(output_path, exist_ok=True)
    
    ydl_opts = {
        'outtmpl': f'{output_path}/%(title)s.%(ext)s',
        'format': 'mp4',
        'merge_output_format': 'mp4'
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        filename = ydl.prepare_filename(info)
        return filename


In [21]:
def extract_unique_frames(video_path, output_dir="frames", interval=10):
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_interval = interval * fps  # convert seconds to frame count
    count = 0
    saved = 0
    prev_frame = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if count % frame_interval == 0:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Compare to previous frame using SSIM
            if prev_frame is not None:
                score, _ = ssim(prev_frame, gray, full=True)
                if score > 0.95:
                    count += 1
                    continue  # skip similar frame

            # Save the frame
            filename = os.path.join(output_dir, f"frame_{count}.jpg")
            cv2.imwrite(filename, frame)
            prev_frame = gray
            saved += 1

        count += 1

    cap.release()
    print(f"✅ Extracted and saved {saved} unique frames to '{output_dir}'.")



In [22]:
youtube_url = "https://www.youtube.com/watch?v=4r4p61Mk84w"
video_path = download_youtube_video_ytdlp(youtube_url)

extract = extract_frames(video_path, output_dir="frames", interval=2)
print(f"Extracted {extract} frames from the video at {video_path}.")


[youtube] Extracting URL: https://www.youtube.com/watch?v=4r4p61Mk84w
[youtube] 4r4p61Mk84w: Downloading webpage
[youtube] 4r4p61Mk84w: Downloading tv client config
[youtube] 4r4p61Mk84w: Downloading tv player API JSON
[youtube] 4r4p61Mk84w: Downloading ios player API JSON
[youtube] 4r4p61Mk84w: Downloading m3u8 information
[info] 4r4p61Mk84w: Downloading 1 format(s): 18
[download] videos\Intro to Industrial Sewing Machines.mp4 has already been downloaded
[download] 100% of   32.40MiB
Extracted 555 frames from the video at videos\Intro to Industrial Sewing Machines.mp4.
